In [1]:
import gensim
import pyemd
import pandas as pd
import numpy as np
from math import sqrt
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from nltk import download
download('stopwords')
stpwrds = stopwords.words('portuguese')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Upload do arquivo de word embedding pré-treinado.
Download: [link](https://drive.google.com/file/d/0B0ZXk88koS2KRDcwcV9IVWFTeUE/view)

In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving pt.bin to pt.bin
Saving pt.bin.syn0.npy to pt.bin.syn0.npy
Saving pt.bin.syn1neg.npy to pt.bin.syn1neg.npy
User uploaded file "pt.bin" with length 4051167 bytes
User uploaded file "pt.bin.syn0.npy" with length 60295280 bytes
User uploaded file "pt.bin.syn1neg.npy" with length 60295280 bytes


In [14]:
model = Word2Vec.load('pt.bin')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


## 1. Implemente uma função que recebe uma notícia e retorna os vetores (word embeddings) das palavras do título dessa notícia a partir dos word embeddings pré-treinados com o gensim.

Método de pré-processamento para remoção de stopwords, números e letras/símbolos.

In [0]:
def preprocess(noticia):
  noticia = noticia.lower().split()
  tokens = [t for t in noticia if t not in stpwrds and t.isalpha()
           and len(t) >= 3]
  return tokens

Definição da notícia que será avaliada a partir dos word embeddings pré-treinados

In [0]:
noticia = 'Votação do pacote de Moro é adiada na Câmara'

In [0]:
def norm12(t, is_neg=False):
  val = model[t] if not is_neg else -1 * model[t]
  return val / sqrt(sum(val ** 2))

def weVetor(noticia):
  tokens = preprocess(noticia)
  valores = [norm12(token) for token in tokens]
  mean = np.array(valores)
  return model.wv.most_similar(positive=[mean.mean(axis=0)])

Vetor de  Word Embeddings para o título da notícia definida acima

In [19]:
wordEmbeddings = weVetor(noticia)
wordEmbeddings

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('votação', 0.6291680335998535),
 ('câmara', 0.5394864678382874),
 ('eleição', 0.49725210666656494),
 ('pe', 0.4964914917945862),
 ('proposta', 0.47250688076019287),
 ('cpi', 0.46748533844947815),
 ('adiada', 0.4647755026817322),
 ('reeleito', 0.45788830518722534),
 ('arquivado', 0.45751550793647766),
 ('alerj', 0.44819629192352295)]

## 2. Implemente uma função que calcula o WMD (Word Mover's Distance) entre duas notícias usando os embeddings das palavras dos respectivos títulos.

Definição das duas notícias que serão comparadas com a notícia utilizada na questão anterior, para fim de demonstrar os valores de distância quando há e quando não há relação entre os textos.

In [20]:
noticia2 = 'Parecer do pacote de Moro deve ser adiado para quarta-feira, diz relator' 
noticia3 = 'Jair Bolsonaro encontra Neymar no Mineirão antes de Brasil x Argentina'

weVetor(noticia2)
weVetor(noticia3)

model.init_sims(replace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [21]:
distancia = model.wmdistance(preprocess(noticia), preprocess(noticia2))
print("Distância: %.4f" % distancia)

Distância: 0.8716


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).
  """Entry point for launching an IPython kernel.


In [22]:
distancia = model.wmdistance(preprocess(noticia), preprocess(noticia3))
print("Distância: %.4f" % distancia)

Distância: 1.2817


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).
  """Entry point for launching an IPython kernel.


## 3. Implemente uma função que possa receber qualquer notícia como entrada e retornar as top-3 notícias mais similares (menos distantes) a ela.

Trazendo o conjunto de notícias utilizados na atividade anterior, podemos garantir o recebimento de qualquer notícia como entrada.

In [0]:
results = pd.read_csv('https://raw.githubusercontent.com/GersonSales/rec-info/master/lab_08/results.csv')
noticias = results.title
tokens = [preprocess(titulo) for titulo in noticias]

model = Word2Vec(tokens, workers=3, size=100)

from gensim.similarities import WmdSimilarity
num_best = 3
instance = WmdSimilarity(tokens, model, num_best=3)

Aqui temos as top-3 notícias que se assemelha à consulta

In [24]:
consultaID = 1
consulta = noticias[consultaID]

print("Consulta:",consulta)
print()

consulta = preprocess(consulta)
sims = instance[consulta]
for i in range(num_best):
    print('sim = %.4f' % sims[i][1])
    print(noticias[sims[i][0]])
    print()

Consulta: Justiça suspende decisão que proibia Forças Armadas de celebrarem golpe de 1964

sim = 1.0000
Justiça suspende decisão que proibia Forças Armadas de celebrarem golpe de 1964

sim = 1.0000
Os clubes que se posicionaram no aniversário do golpe militar

sim = 1.0000
Celebrar o golpe representa uma derrota para a democracia

